In [1]:
import numpy as np
import pandas as pd
from scipy import linalg
from math import sqrt
from tabulate import tabulate

In [5]:
np.set_printoptions(formatter={'float_kind':"{:.8f}".format})
pd.options.display.float_format = '{:.8f}'.format

In [4]:
def pretty_view(a):
    pret=pd.DataFrame(data=a, index=list(range(1, a.shape[0]+1)), columns=list(range(1, a.shape[1]+1)))
    display(pret)

In [2]:
def count_b(A, x):  # считаем b для последующих вычислений
    b = np.dot(A, x)
    return b

In [9]:
def sqrt_method(A):  # Находим L из А
    L = np.zeros((A.shape[0], A.shape[0]))
    for i in range(A.shape[0]):
        for j in range(i):
            temp = 0
            for k in range(j):
                temp += L[i][k] * L[j][k]
            L[i][j] = (A[i][j] - temp) / L[j][j]
        temp = A[i][i]
        for k in range(i):
            temp -= L[i][k] * L[i][k]
        L[i][i] = sqrt(temp)
    return L

In [8]:
def solve(L, b):  # решаем регуляризационную систему, находим x, y
    y = np.linalg.solve(L, b)
    x = np.linalg.solve(L.transpose(), y)
    return x

In [6]:
def rotation_method(A, b):  # реализация метода вращений
    q = np.column_stack([A, b]) # собирает из колонок матрицу, колонки соединяются в том порядке, который указан
    for i in range(q.shape[0] - 1):
        for j in range(i + 1, q.shape[0]):
            c = q[i, i] / (q[i, i] ** 2 + q[j, i] ** 2) ** (1/2)
            s = q[j, i] / (q[i, i] ** 2 + q[j, i] ** 2) ** (1/2)
            tmp = q[i, :] * c + q[j, :] * s
            q[j, :] = q[i, :] * -s + q[j, :] * c
            q[i, :] = tmp
    x = np.linalg.solve(q[:, :-1], q[:, -1])
    return x

In [17]:
table=[['Порядок', "x - x_rot", 'x - x_sqrt']]
for n in (3, 4, 5, 10):
    row=[]
    row.append(n)
    A = linalg.hilbert(n)
    x = np.random.uniform(0, 100, size=A.shape[0])
    b = count_b(A, x)
    x_rot = rotation_method(A, b)
    x_rot = np.linalg.norm(x - x_rot)
    row.append(x_rot)
    x_sqrt= np.linalg.norm(x - solve(sqrt_method(A), b))
    row.append(x_sqrt)
    table.append(row)

In [18]:
print(tabulate(table, tablefmt='fancy_grid', numalign="right"))

╒═════════╤════════════════════════╤════════════════════════╕
│ Порядок │ x - x_rot              │ x - x_sqrt             │
├─────────┼────────────────────────┼────────────────────────┤
│ 3       │ 3.911434501196473e-12  │ 2.602820459648654e-12  │
├─────────┼────────────────────────┼────────────────────────┤
│ 4       │ 7.604680583663828e-11  │ 8.593673709481708e-11  │
├─────────┼────────────────────────┼────────────────────────┤
│ 5       │ 1.5124006574256817e-10 │ 3.4165585617978954e-10 │
├─────────┼────────────────────────┼────────────────────────┤
│ 10      │ 0.029068267913885528   │ 0.00022259959330804552 │
╘═════════╧════════════════════════╧════════════════════════╛
